# Lab: Trade-offs and Star Schema Design

## 1. Prerequisites & Setup
*   **Context:** This lab explores the practical trade-offs between normalized and denormalized designs.
*   **Goal:** Analyze query performance differences and design a star schema.
*   **Concept Review:** Ensure you have read `w03_l06_concept_denormalization.md`.

### Environment Setup
Run this block first to set up the environment and create sample data.

In [ ]:
# Setup: Run this cell first (required for Colab)
# NOTE: Run cells in order. Variables from earlier sections are used later.
!pip install -q pandas duckdb mermaid-py

In [ ]:
import pandas as pd
import duckdb
import time
from mermaid import Mermaid

pd.set_option('display.max_columns', None)
print("Setup complete! Ready for trade-off analysis.")

---

## 2. The Scenario: E-Commerce Analytics
You're a Data Scientist at an e-commerce company. The operational database is normalized (from Lesson 5). Now the business wants a dashboard showing:

1.  Total revenue by product category
2.  Top customers by purchase volume
3.  Monthly sales trends

Your job: decide whether to query the normalized tables directly or create a denormalized analytical layer.

### Create the Normalized Tables

In [ ]:
# Initialize DuckDB in-memory database
conn = duckdb.connect(':memory:')

# Create normalized tables
conn.execute("""
    CREATE TABLE customers (
        customer_id INTEGER PRIMARY KEY,
        name VARCHAR,
        email VARCHAR,
        city VARCHAR,
        segment VARCHAR  -- 'Consumer', 'Corporate', 'Home Office'
    )
""")

conn.execute("""
    CREATE TABLE products (
        product_id VARCHAR PRIMARY KEY,
        name VARCHAR,
        category VARCHAR,
        unit_price DECIMAL(10,2)
    )
""")

conn.execute("""
    CREATE TABLE orders (
        order_id INTEGER PRIMARY KEY,
        order_date DATE,
        customer_id INTEGER REFERENCES customers(customer_id)
    )
""")

conn.execute("""
    CREATE TABLE order_items (
        order_id INTEGER REFERENCES orders(order_id),
        product_id VARCHAR REFERENCES products(product_id),
        quantity INTEGER,
        PRIMARY KEY (order_id, product_id)
    )
""")

print("Normalized schema created!")

### Populate with Sample Data

In [ ]:
import random
from datetime import date, timedelta

# Seed for reproducibility
random.seed(42)

# Insert customers
customers = [
    (i, f"Customer_{i}", f"cust{i}@email.com",
     random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']),
     random.choice(['Consumer', 'Corporate', 'Home Office']))
    for i in range(1, 1001)
]
conn.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?)", customers)

# Insert products
categories = ['Electronics', 'Clothing', 'Home & Garden', 'Sports', 'Books']
products = [
    (f"P{i:04d}", f"Product_{i}", random.choice(categories), round(random.uniform(5, 500), 2))
    for i in range(1, 201)
]
conn.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", products)

# Insert orders and order_items
start_date = date(2023, 1, 1)
order_id = 1

for _ in range(5000):  # 5000 orders
    order_date = start_date + timedelta(days=random.randint(0, 729))  # 2 years of data
    customer_id = random.randint(1, 1000)
    conn.execute("INSERT INTO orders VALUES (?, ?, ?)", (order_id, order_date, customer_id))

    # Each order has 1-5 items
    num_items = random.randint(1, 5)
    selected_products = random.sample(range(1, 201), num_items)
    for prod_num in selected_products:
        product_id = f"P{prod_num:04d}"
        quantity = random.randint(1, 10)
        conn.execute("INSERT INTO order_items VALUES (?, ?, ?)", (order_id, product_id, quantity))

    order_id += 1

print(f"Inserted 1000 customers, 200 products, 5000 orders")
print(f"Total order_items: {conn.execute('SELECT COUNT(*) FROM order_items').fetchone()[0]}")

<details>
<summary>Expected Output</summary>

~~~text
Inserted 1000 customers, 200 products, 5000 orders
Total order_items: ~15000 (varies due to random 1-5 items per order)
~~~

</details>

---

## 3. Step 1: Query the Normalized Schema

Let's run the business queries on the normalized tables.

> **Performance Note:** With only ~15K rows, both normalized and star schema queries will complete in milliseconds. DuckDB is highly optimized and the performance difference at this scale is minimal. In production systems with **millions of rows**, the star schema advantage becomes significant (10x+ speedup is common). We use this small dataset to demonstrate the *pattern*, not to prove performance claims.

### Query 1: Revenue by Category

In [ ]:
query_normalized = """
SELECT
    p.category,
    SUM(oi.quantity * p.unit_price) AS total_revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.category
ORDER BY total_revenue DESC
"""

start = time.time()
result = conn.execute(query_normalized).df()
normalized_time = time.time() - start

print(f"Query time (normalized): {normalized_time:.4f} seconds")
print(result)

# Note: For more reliable timing with small queries, you could run multiple iterations:
# times = [timeit.timeit(lambda: conn.execute(query_normalized).df(), number=1) for _ in range(100)]
# print(f"Average time: {sum(times)/len(times):.4f} seconds")

<details>
<summary>Expected Output</summary>

~~~text
Query time (normalized): ~0.01-0.05 seconds
        category  total_revenue
0    Electronics      XXXXXX.XX
1       Clothing      XXXXXX.XX
2  Home & Garden      XXXXXX.XX
3         Sports      XXXXXX.XX
4          Books      XXXXXX.XX
~~~

</details>

### Query 2: Top 10 Customers by Revenue

In [ ]:
query_top_customers = """
SELECT
    c.customer_id,
    c.name,
    c.segment,
    SUM(oi.quantity * p.unit_price) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
GROUP BY c.customer_id, c.name, c.segment
ORDER BY total_spent DESC
LIMIT 10
"""

start = time.time()
result = conn.execute(query_top_customers).df()
normalized_time_2 = time.time() - start

print(f"Query time (normalized): {normalized_time_2:.4f} seconds")
print(result)

<details>
<summary>Expected Output</summary>

~~~text
Query time (normalized): ~0.01-0.05 seconds
   customer_id          name      segment  total_spent
0          XXX  Customer_XXX  XXXXXXXXXX     XXXXX.XX
1          XXX  Customer_XXX  XXXXXXXXXX     XXXXX.XX
...
~~~

</details>

---

## 4. Step 2: Create a Denormalized Fact Table

Now let's create a star schema with a pre-joined fact table.

### Design the Star Schema

In [ ]:
# Note: dim_date will be added in Exercise 2
Mermaid("""
erDiagram
    fact_sales {
        int sale_id PK
        int order_id
        int date_key FK
        int customer_id FK
        varchar product_id FK
        int quantity
        decimal unit_price
        decimal line_total
    }
    dim_date {
        int date_key PK
        date full_date
        int year
        int month
        varchar month_name
        int quarter
    }
    dim_customer {
        int customer_id PK
        varchar name
        varchar city
        varchar segment
    }
    dim_product {
        varchar product_id PK
        varchar name
        varchar category
    }

    fact_sales }|--|| dim_date : "date_key"
    fact_sales }|--|| dim_customer : "customer_id"
    fact_sales }|--|| dim_product : "product_id"
""")

### Build the Fact Table

In [ ]:
# Create fact_sales with pre-computed values
conn.execute("""
    CREATE TABLE fact_sales AS
    SELECT
        ROW_NUMBER() OVER () AS sale_id,
        o.order_id,
        o.order_date,
        o.customer_id,
        oi.product_id,
        oi.quantity,
        p.unit_price,
        (oi.quantity * p.unit_price) AS line_total,
        EXTRACT(YEAR FROM o.order_date) AS year,
        EXTRACT(MONTH FROM o.order_date) AS month
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
""")

# Create dimension tables (simplified from normalized tables)
conn.execute("""
    CREATE TABLE dim_customer AS
    SELECT customer_id, name, city, segment
    FROM customers
""")

conn.execute("""
    CREATE TABLE dim_product AS
    SELECT product_id, name, category
    FROM products
""")

print("Star schema created!")
print(f"fact_sales rows: {conn.execute('SELECT COUNT(*) FROM fact_sales').fetchone()[0]}")

<details>
<summary>Expected Output</summary>

~~~text
Star schema created!
fact_sales rows: ~15000
~~~

</details>

---

## 5. Step 3: Compare Query Performance

### Revenue by Category (Star Schema)

In [ ]:
query_star = """
SELECT
    dp.category,
    SUM(fs.line_total) AS total_revenue
FROM fact_sales fs
JOIN dim_product dp ON fs.product_id = dp.product_id
GROUP BY dp.category
ORDER BY total_revenue DESC
"""

start = time.time()
result = conn.execute(query_star).df()
star_time = time.time() - start

print(f"Query time (star schema): {star_time:.4f} seconds")
print(f"Query time (normalized):  {normalized_time:.4f} seconds")
print(f"Speedup: {normalized_time/star_time:.2f}x" if star_time > 0 else "N/A")
print(result)

<details>
<summary>Expected Output</summary>

~~~text
Query time (star schema): ~0.005-0.02 seconds
Query time (normalized):  ~0.01-0.05 seconds
Speedup: 1.5-3x
~~~

With larger datasets, the difference becomes more pronounced.

</details>

### Top Customers (Star Schema)

In [ ]:
query_star_customers = """
SELECT
    dc.customer_id,
    dc.name,
    dc.segment,
    SUM(fs.line_total) AS total_spent
FROM fact_sales fs
JOIN dim_customer dc ON fs.customer_id = dc.customer_id
GROUP BY dc.customer_id, dc.name, dc.segment
ORDER BY total_spent DESC
LIMIT 10
"""

start = time.time()
result = conn.execute(query_star_customers).df()
star_time_2 = time.time() - start

print(f"Query time (star schema): {star_time_2:.4f} seconds")
print(f"Query time (normalized):  {normalized_time_2:.4f} seconds")

---

## 6. Step 4: Analyze the Trade-offs

### Storage Comparison

In [ ]:
# Compare storage (approximate via row counts and columns)
normalized_size = conn.execute("""
    SELECT
        (SELECT COUNT(*) FROM customers) AS customers,
        (SELECT COUNT(*) FROM products) AS products,
        (SELECT COUNT(*) FROM orders) AS orders,
        (SELECT COUNT(*) FROM order_items) AS order_items
""").df()

star_size = conn.execute("""
    SELECT
        (SELECT COUNT(*) FROM dim_customer) AS dim_customer,
        (SELECT COUNT(*) FROM dim_product) AS dim_product,
        (SELECT COUNT(*) FROM fact_sales) AS fact_sales
""").df()

print("Normalized schema row counts:")
print(normalized_size)
print("\nStar schema row counts:")
print(star_size)

<details>
<summary>Expected Output</summary>

~~~text
Normalized schema row counts:
   customers  products  orders  order_items
0       1000       200    5000        ~15000

Star schema row counts:
   dim_customer  dim_product  fact_sales
0          1000          200      ~15000
~~~

Row counts are similar, but `fact_sales` has more columns (redundant `unit_price`, pre-computed `line_total`, `year`, `month`).

**Column comparison:**
- Normalized: 4 tables × ~4 columns each = ~16 columns total
- Star: 3 tables, but `fact_sales` alone has 10 columns (including redundant data)

The storage overhead is the *price* of denormalization. Worth it when queries are frequent and updates are rare.

</details>

### Update Scenario Analysis

In [ ]:
# Scenario: Product P0001's price changes from $X to $Y
# How many rows need updating?

print("If Product P0001's price changes:")
print(f"  Normalized: 1 row (products table)")

p0001_orders = conn.execute("""
    SELECT COUNT(*) FROM fact_sales WHERE product_id = 'P0001'
""").fetchone()[0]
print(f"  Star Schema: {p0001_orders} rows (fact_sales) OR keep historical prices")

<details>
<summary>Discussion</summary>

This is the key trade-off:
*   **Normalized:** One update, always current, no history
*   **Star Schema:** Many updates needed, BUT often we *want* historical prices preserved

In analytics, keeping the original transaction price is often correct business logic ("What did we actually charge?").

</details>

---

## 7. Your Turn! (Exercises)

### Exercise 1: Monthly Sales Trend Query
**Task:** Write both normalized and star schema queries to get monthly revenue for 2024.

In [ ]:
# TODO: Normalized version (join orders, order_items, products)
query_monthly_normalized = """
-- Your query here
"""

# TODO: Star schema version (just fact_sales with year/month columns)
query_monthly_star = """
-- Your query here
"""

<details>
<summary>Expected Output</summary>

~~~python
# Normalized
query_monthly_normalized = """
SELECT
    EXTRACT(YEAR FROM o.order_date) AS year,
    EXTRACT(MONTH FROM o.order_date) AS month,
    SUM(oi.quantity * p.unit_price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE EXTRACT(YEAR FROM o.order_date) = 2024
GROUP BY 1, 2
ORDER BY 1, 2
"""

# Star Schema
query_monthly_star = """
SELECT
    year,
    month,
    SUM(line_total) AS revenue
FROM fact_sales
WHERE year = 2024
GROUP BY year, month
ORDER BY year, month
"""
~~~

The star schema query is simpler and doesn't require extracting date parts on the fly.

</details>

### Exercise 2: Add a Time Dimension

> **💡 Why Create a Separate `dim_date` Table?**
>
> **Current approach:** We extracted `year` and `month` directly in the fact table (lines 295-296). This works for simple queries.
>
> **Production pattern:** Create a separate `dim_date` table because:
> 1. **Rich attributes**: Month names, quarters, weekends, holidays, fiscal periods
> 2. **Pre-computed**: Calculate once, not on every query (faster for millions of rows)
> 3. **Business logic**: Fiscal years, custom calendars, holiday rules
> 4. **Consistency**: Same date logic across all fact tables in your warehouse
> 5. **Flexibility**: Add new attributes (e.g., "is_holiday") without touching fact tables
>
> **Example benefit:** Instead of complex date math in every query, just:
> ```sql
> WHERE dd.is_weekend = FALSE AND dd.is_holiday = FALSE
> ```

**Task:** Build a `dim_date` dimension table following the industry-standard star schema pattern.

---

#### **Step 2a: Generate All Dates**

> **Why Generate All Dates? Can't we just use dates from the `orders` table?**
>
> **Answer:** No! `dim_date` must contain **every date**, not just dates with orders. Here's why:
>
> **Problem with using only order dates:**
> - **Missing dates:** If no orders on 2024-12-25, Christmas won't exist in `dim_date`
> - **Broken charts:** Revenue reports will have gaps (can't show "$0 on Dec 25")
> - **Can't analyze absences:** Can't find "which weekends had zero sales?" if those dates don't exist
> - **Future planning:** Can't show next month's calendar if orders don't exist yet
>
> **Think of `dim_date` as a calendar on the wall** - it has every date, whether events happened or not.
>
> **Example benefit:**
> ```sql
> -- Show daily sales for January 2024 (including days with $0)
> SELECT dd.full_date, COALESCE(SUM(fs.line_total), 0) as revenue
> FROM dim_date dd
> LEFT JOIN fact_sales fs ON dd.date_key = fs.date_key
> WHERE dd.year = 2024 AND dd.month = 1
> ```
> This only works if dim_date has ALL days of January, not just days with orders!

First, we need a table with one row per date. DuckDB provides a `range()` function for this.

In [ ]:
# TODO: Create a date series for 2023-2024
# Hint: DuckDB's range(start_date, end_date, interval) generates a series

conn.execute("""
    CREATE TABLE dim_date_basic AS
    SELECT CAST(range AS DATE) AS full_date
    FROM range(DATE '2023-01-01', DATE '2025-01-01', INTERVAL 1 DAY)
""")

# Check what we created
print("Sample dates:")
print(conn.execute("SELECT * FROM dim_date_basic LIMIT 5").df())
print(f"\nTotal dates: {conn.execute('SELECT COUNT(*) FROM dim_date_basic').fetchone()[0]}")

<details>
<summary>Expected Output</summary>

~~~text
Sample dates:
   full_date
0 2023-01-01
1 2023-01-02
2 2023-01-03
3 2023-01-04
4 2023-01-05

Total dates: 730 (2 years of dates)
~~~

</details>

---

#### **Step 2b: Extract Date Components**

Now let's add useful date attributes. SQL provides two main functions for working with dates:

| Function | Purpose | Example |
|:---|:---|:---|
| `EXTRACT(part FROM date)` | Get numeric part of date | `EXTRACT(YEAR FROM '2024-03-15')` → 2024 |
| `STRFTIME(date, format)` | Format date as string | `STRFTIME('2024-03-15', '%B')` → 'March' |

**Common EXTRACT parts:** `YEAR`, `MONTH`, `DAY`, `QUARTER`, `DAYOFWEEK` (0=Sunday, 6=Saturday)

**Common STRFTIME formats:**
- `'%Y'` = 4-digit year (2024)
- `'%m'` = 2-digit month (03)
- `'%B'` = Full month name (March)
- `'%Y%m%d'` = YYYYMMDD format (20240315) - common for date keys

In [ ]:
# TODO: Add year, month, and month_name columns
# Hint: Use EXTRACT for numbers, STRFTIME for text

conn.execute("""
    CREATE TABLE dim_date_enriched AS
    SELECT
        full_date,
        EXTRACT(YEAR FROM full_date) AS year,
        EXTRACT(MONTH FROM full_date) AS month,
        STRFTIME(full_date, '%B') AS month_name
        -- We'll add more columns in the next step
    FROM dim_date_basic
""")

print(conn.execute("SELECT * FROM dim_date_enriched LIMIT 5").df())

<details>
<summary>Expected Output</summary>

~~~text
   full_date  year  month month_name
0 2023-01-01  2023      1    January
1 2023-01-02  2023      1    January
2 2023-01-03  2023      1    January
3 2023-01-04  2023      1    January
4 2023-01-05  2023      1    January
~~~

</details>

---

#### **Step 2c: Add Business-Useful Attributes**

Let's add more attributes that analysts commonly need:

In [ ]:
# TODO: Add quarter, day_of_week, and is_weekend
# Hint: EXTRACT(QUARTER ...), EXTRACT(DAYOFWEEK ...) where 0=Sunday, 6=Saturday
# Hint: Use CASE WHEN for is_weekend

conn.execute("""
    CREATE TABLE dim_date AS
    SELECT
        CAST(STRFTIME(full_date, '%Y%m%d') AS INTEGER) AS date_key,  -- e.g., 20240315
        full_date,
        EXTRACT(YEAR FROM full_date) AS year,
        EXTRACT(MONTH FROM full_date) AS month,
        STRFTIME(full_date, '%B') AS month_name,
        EXTRACT(QUARTER FROM full_date) AS quarter,  -- 1, 2, 3, or 4
        EXTRACT(DAYOFWEEK FROM full_date) AS day_of_week,  -- 0=Sun, 1=Mon, ..., 6=Sat
        CASE
            WHEN EXTRACT(DAYOFWEEK FROM full_date) IN (0, 6) THEN TRUE
            ELSE FALSE
        END AS is_weekend
    FROM dim_date_basic
""")

print("Sample dim_date rows:")
print(conn.execute("SELECT * FROM dim_date LIMIT 10").df())

<details>
<summary>Expected Output</summary>

~~~text
Sample dim_date rows:
   date_key   full_date  year  month month_name  quarter  day_of_week  is_weekend
0  20230101  2023-01-01  2023      1    January        1            0        True  (Sunday)
1  20230102  2023-01-02  2023      1    January        1            1       False  (Monday)
2  20230103  2023-01-03  2023      1    January        1            2       False
3  20230104  2023-01-04  2023      1    January        1            3       False
4  20230105  2023-01-05  2023      1    January        1            4       False
5  20230106  2023-01-06  2023      1    January        1            5       False
6  20230107  2023-01-07  2023      1    January        1            6        True  (Saturday)
...
~~~

</details>

---

#### **Step 2d: Link Fact Table to dim_date**

Now we need to add a `date_key` column to `fact_sales` and populate it.

In [ ]:
# Step 1: Add the column
conn.execute("""
    ALTER TABLE fact_sales ADD COLUMN date_key INTEGER;
""")

# Step 2: Populate it by converting order_date to YYYYMMDD format
# TODO: Update fact_sales to set date_key
# Hint: Use the same STRFTIME format as in dim_date

conn.execute("""
    UPDATE fact_sales
    SET date_key = CAST(STRFTIME(order_date, '%Y%m%d') AS INTEGER);
""")

print("Updated fact_sales with date_key:")
print(conn.execute("SELECT sale_id, order_date, date_key FROM fact_sales LIMIT 5").df())

<details>
<summary>Expected Output</summary>

~~~text
Updated fact_sales with date_key:
   sale_id  order_date  date_key
0        1  2023-05-12  20230512
1        2  2023-05-12  20230512
2        3  2023-05-12  20230512
3        4  2024-01-08  20240108
4        5  2024-01-08  20240108
~~~

</details>

---

#### **Step 2e: Query with Rich Date Attributes**

Now we can use all the pre-computed date attributes!

In [ ]:
# Example 1: Revenue by quarter
print("Revenue by quarter:")
print(conn.execute("""
    SELECT
        dd.year,
        dd.quarter,
        SUM(fs.line_total) as revenue
    FROM fact_sales fs
    JOIN dim_date dd ON fs.date_key = dd.date_key
    GROUP BY dd.year, dd.quarter
    ORDER BY dd.year, dd.quarter
""").df())

# Example 2: Weekend vs. weekday sales
print("\nWeekend vs. Weekday sales:")
print(conn.execute("""
    SELECT
        CASE WHEN dd.is_weekend THEN 'Weekend' ELSE 'Weekday' END as period,
        COUNT(*) as num_sales,
        SUM(fs.line_total) as total_revenue
    FROM fact_sales fs
    JOIN dim_date dd ON fs.date_key = dd.date_key
    GROUP BY dd.is_weekend
""").df())

<details>
<summary>Expected Output</summary>

~~~text
Revenue by quarter:
   year  quarter      revenue
0  2023        1  XXXXXXXX.XX
1  2023        2  XXXXXXXX.XX
2  2023        3  XXXXXXXX.XX
3  2023        4  XXXXXXXX.XX
4  2024        1  XXXXXXXX.XX
5  2024        2  XXXXXXXX.XX
6  2024        3  XXXXXXXX.XX
7  2024        4  XXXXXXXX.XX

Weekend vs. Weekday sales:
    period  num_sales  total_revenue
0  Weekday      ~XXXX    XXXXXXXX.XX
1  Weekend      ~XXXX    XXXXXXXX.XX
~~~

</details>

---

#### **🎯 Key Takeaway**

With `dim_date`, you now have:
- ✅ Pre-computed date attributes (no `EXTRACT()` in every query)
- ✅ Business-friendly names (`month_name` instead of numbers)
- ✅ Complex date logic (weekends, quarters) in simple WHERE clauses
- ✅ Consistent date handling across all fact tables

**Industry Standard:** Every data warehouse has a `dim_date` table. You've just built one!

---

#### **Optional Challenge: Add Fiscal Year**

Many companies have fiscal years that don't match calendar years (e.g., fiscal year starts April 1).

In [ ]:
# TODO: Add a fiscal_year column where fiscal year starts on July 1
# Hint: If month >= 7, fiscal_year = calendar_year + 1, else fiscal_year = calendar_year

# Example solution:
conn.execute("""
    ALTER TABLE dim_date ADD COLUMN fiscal_year INTEGER;
""")
conn.execute("""
    UPDATE dim_date
    SET fiscal_year = CASE
        WHEN month >= 7 THEN year + 1
        ELSE year
    END;
""")

print(conn.execute("""
    SELECT full_date, year, month, fiscal_year
    FROM dim_date
    WHERE month IN (6, 7)
    LIMIT 6
""").df())

<details>
<summary>Expected Output</summary>

~~~text
   full_date  year  month  fiscal_year
0 2023-06-29  2023      6         2023  (still FY 2023)
1 2023-06-30  2023      6         2023
2 2023-07-01  2023      7         2024  (FY 2024 starts!)
3 2023-07-02  2023      7         2024
4 2024-06-29  2024      6         2024
5 2024-06-30  2024      6         2024
~~~

Shows the fiscal year transition on July 1.

</details>

### Exercise 3: When to Normalize vs. Denormalize
**Task:** For each scenario, recommend normalized or denormalized design and explain why.

| Scenario | Your Recommendation | Why? |
| :--- | :--- | :--- |
| Banking transaction system | | |
| Marketing dashboard | | |
| IoT sensor data for real-time monitoring | | |
| ML feature store for model training | | |
| User profile management | | |

<details>
<summary>Expected Answers</summary>

| Scenario | Recommendation | Why |
| :--- | :--- | :--- |
| Banking transaction system | **Normalized** | Data integrity critical, ACID compliance, frequent writes |
| Marketing dashboard | **Denormalized (Star)** | Read-heavy, aggregations, historical snapshots |
| IoT sensor data for real-time monitoring | **Denormalized (Time-series)** | Append-only writes (no updates), time-range queries, often uses specialized time-series DBs (InfluxDB, TimescaleDB) |
| ML feature store for model training | **Denormalized** | Pre-computed features, read-heavy during training |
| User profile management | **Normalized** | Frequent updates, data consistency important |

</details>

---

## 8. Summary
You have successfully:
1.  Built both **normalized** and **star schema** versions of the same data.
2.  Compared **query complexity** and **performance**.
3.  Analyzed **storage** and **update** trade-offs.
4.  Practiced designing **dimension tables** for analytical workloads.

**Key Takeaway:** Neither approach is universally better. Match your schema design to your workload:
*   **OLTP (transactional):** Normalize for integrity
*   **OLAP (analytical):** Denormalize for speed